In [1]:
import csv
from argparse import ArgumentParser

import numpy as np
import pandas as pd
from pathlib import Path
import mysql.connector

In [2]:
root = Path(r"D:\recording\run00063_2024-05-16")
results_dir = root / 'kilosort4'

# One value per cluster
camps = pd.read_csv(results_dir / 'cluster_Amplitude.tsv', sep='\t')['Amplitude'].values
clu_labels = pd.read_csv(results_dir / 'cluster_KSLabel.tsv', sep='\t')['KSLabel'].values
contam_pct = pd.read_csv(results_dir / 'cluster_ContamPct.tsv', sep='\t')['ContamPct'].values
chan_map =  np.load(results_dir / 'channel_map.npy')
templates =  np.load(results_dir / 'templates.npy')
chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
chan_best = chan_map[chan_best]

# One value per spike
amplitudes = np.load(results_dir / 'amplitudes.npy')
st = np.load(results_dir / 'spike_times.npy')
clu = np.load(results_dir / 'spike_clusters.npy')

sr = 20_000

subfolders, n_samples = [], []
with open(root / 'concatenated.csv', newline='') as f:
    csvfile = csv.reader(f)
    _ = next(csvfile)  # skip header
    for (subfolder, n) in csvfile:
        subfolders.append(subfolder)
        n_samples.append(int(n))
recordings_df = pd.DataFrame(dict(subfolder=subfolders, n_samples=n_samples))

In [3]:
recordings_df['cum_samples'] = np.append(0, np.cumsum(recordings_df.n_samples[:-1]))
recordings_df['notefile'] = notefiles = [f.replace('experiment', 'notes') + '.txt' for f in subfolders]
nf_str = ', '.join(f'"{f}"' for f in notefiles)

In [4]:
db = mysql.connector.connect(
    user='xper_rw',
    password='up2nite',
    host='172.30.6.54',
    database='SpikeData',
    client_flags=[mysql.connector.constants.ClientFlag.LOCAL_FILES],
    allow_local_infile=True,
)

In [7]:
recordings_df = rdf.join(recordings_df.set_index('notefile'), on='notefile', how='inner').reset_index().loc[:, ['recording_id', 'cum_samples']]

In [33]:
curs.execute(f"""
    SELECT    recording_id, trial_id, start_sample, stop_sample 
    FROM      Trials INNER JOIN Recordings USING (recording_id)
    WHERE     recording_id IN ({rstr})
    ORDER BY  recording_id, start_sample
""")
trials = pd.DataFrame.from_records(curs.fetchall(), columns=['recording_id', 'trial_id', 'start_sample', 'stop_sample'])
trials = trials.join(recordings_df.set_index('recording_id'), on='recording_id')
trials.start_sample += trials_df.cum_samples
trials.stop_sample += trials_df.cum_samples
trials.set_index(pd.Series(np.arange(len(trials)), name='trial_idx'), inplace=True)

In [34]:
trials

,recording_id,trial_id,start_sample,stop_sample,cum_samples
trial_idx,,,,,
0,1715888363673144,1715888365270000,56016,186652,0
1,1715888363673144,1715888373916000,228833,356281,0
2,1715888363673144,1715888382423000,399018,525988,0
3,1715888363673144,1715888390887000,568502,733951,0
4,1715888363673144,1715888401281000,776355,962325,0
...,...,...,...,...,...
1679,1715900310673347,1715900827801000,245423453,245515090,235057024
1680,1715900310673347,1715900834494000,245557606,245650719,235057024
1681,1715900310673347,1715900841263000,245692788,245809426,235057024


In [63]:
spikes = pd.DataFrame(dict(
    spike_sample=st, 
    cluster_idx=clu,
    amplitude=amplitudes,
    trial_idx=np.searchsorted(trials.start_sample, st) - 1,
))

In [67]:
t = 399018 + 2
np.searchsorted(trials.start_sample, t)

3

In [62]:
a = trials.start_sample.values
np.all(a[:-1] <= a[1:])

True

In [69]:
trial_spks = spikes.join(trials, how='inner', on='trial_idx')
# trial_spks = trial_spks.query('start_sample < spike_sample < stop_sample')

In [70]:
trial_spks

,spike_sample,cluster_idx,amplitude,trial_idx,recording_id,trial_id,start_sample,stop_sample,cum_samples
497,56025,11,15.801606,0,1715888363673144,1715888365270000,56016,186652,0
498,56088,30,17.841702,0,1715888363673144,1715888365270000,56016,186652,0
499,56101,38,12.859563,0,1715888363673144,1715888365270000,56016,186652,0
500,56138,13,18.313335,0,1715888363673144,1715888365270000,56016,186652,0
501,56296,13,17.115410,0,1715888363673144,1715888365270000,56016,186652,0
...,...,...,...,...,...,...,...,...,...
1552646,246199375,34,11.784316,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1552647,246199485,20,14.534501,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1552648,246199977,34,12.978854,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1552649,246200090,33,25.646690,1683,1715900310673347,1715900856827000,246004124,246067428,235057024


In [71]:
trial_spks.query('start_sample < spike_sample < stop_sample')

,spike_sample,cluster_idx,amplitude,trial_idx,recording_id,trial_id,start_sample,stop_sample,cum_samples
497,56025,11,15.801606,0,1715888363673144,1715888365270000,56016,186652,0
498,56088,30,17.841702,0,1715888363673144,1715888365270000,56016,186652,0
499,56101,38,12.859563,0,1715888363673144,1715888365270000,56016,186652,0
500,56138,13,18.313335,0,1715888363673144,1715888365270000,56016,186652,0
501,56296,13,17.115410,0,1715888363673144,1715888365270000,56016,186652,0
...,...,...,...,...,...,...,...,...,...
1550985,246067063,28,14.629146,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1550986,246067133,16,15.688227,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1550987,246067255,30,20.849121,1683,1715900310673347,1715900856827000,246004124,246067428,235057024
1550988,246067283,49,13.103801,1683,1715900310673347,1715900856827000,246004124,246067428,235057024


In [12]:
i = 1
spks=spikes_df[spikes_df.folder_idx == i].drop('folder_idx', axis=1)
trials=trials_df[trials_df.folder_idx == i].drop('folder_idx', axis=1)

In [16]:
trials['trial_idx'] = np.arange(len(trials))
spks['trial_idx'] = np.searchsorted(trials.start_sample, spks.spike_time) - 1
trial_spks = spks.join(trials, how='inner', on='trial_idx', lsuffix='l_', rsuffix='r_')
trial_spks.query('start_sample < spike_time <= stop_sample')
trial_spks['spike_time'] = (trial_spks.spike_time - trial_spks.start_sample) / sr
trial_spks = trial_spks.loc[:, ['trial_id', 'cluster_idx', 'spike_time', 'amplitude']]
trial_spks.rename(columns=dict(cluster_idx='channel', spike_time='time'), inplace=True)